<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install chardet
%pip install ftfy

In [19]:
from openai import OpenAI
import os
import uuid
import json
from ftfy import fix_encoding

client = OpenAI()

In [16]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)
    print(detect_encoding('books/' + filename))

print(f'Books in list: {len(filenames)}')

utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
Books in list: 45


In [14]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    return encoding


In [23]:
#generate quetsions to the "answers" extracted from the text
def answer_gpt(answer):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are assisting in the generation of training data for fine-tuning. You will receive a chunk of text, and will respond with a short casually phrased question to which the chunk of text you received would be an expected answer. The person who generated the answer is Alan Watt's and often he will give a response that answers a question only indirectly. The question should not exactly contain the subject matter of the answer. The question you create should be one to which the answer would be a correct indirect or metaphorical answer." 
            },
            {
                "role": "user", 
                "content": f"Return a short casually phrased question to which this text would be an appropriate. The question should not reference the core subject matter of the answer in an overt way. : {answer}"
            }
        ]
    )
    return response.choices[0].message.content.strip()

In [24]:
#split chunks into sections that could reasonably be the answer to a question
def chunk_gpt(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "
            },
            {
                "role": "user", 
                "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"
            }
        ]
    )
    return response.choices[0].message.content.strip().split('\n') 

In [30]:
#loop through the chunks of a book
def process_book_chunks(text_chunks):
    i = 0
    for chunk in text_chunks:
        i += 1
        chunk = fix_encoding(chunk)
        #skip the first and last pages which are usually credits and other misellaneous content
        if i > len(text_chunks)-8 or i < 8:
            continue

        #limiting requests for testing purposes
        if i > 9:
            continue

        processed_answers = chunk_gpt(chunk)
        for answer in processed_answers:
            
            #remove blanks
            if len(answer) < 30:
                continue

            question = answer_gpt(answer)

            json_obj = { 
                "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.", 
                "input" : question, 
                "output" : answer 
            }
            print(json_obj)
            processed_json.append(json_obj)

In [31]:
#process books into chunks of characters

chunk_size = 5000
processed_json = []  

for i in range (1,2):
    encoding = detect_encoding(f'books/{filenames[i]}')
    with open(f'books/{filenames[i]}', 'r', encoding=encoding, errors='replace') as file:
        content = file.read().replace('\n','')
        text_chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        print(f'number of chunks: {len(text_chunks)}')
        process_book_chunks(text_chunks)



print(json.dumps(processed_json, indent=4))

number of chunks: 197
{'instruction': 'You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.', 'input': 'Why do people have such mixed feelings about certain public figures?', 'output': 'Often upstaging Watts’ more learned offerings is a dualistic dialogue of idolatry and iconoclasm reverberating in and resounding through conversations about him. Here one set of voices articulates and recapitulates Watts’ “popular guru” reputation while an iconoclastic and skeptical constituency undercuts the adulation by dismissal of him in summary fashion.'}
{'instruction': 'You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.', 'input': "How do you make lesser-known parts of someone's life more noticeable?", 'output': 'In the Academy is thus an attempt to add tone and volume to some routinely muffled aspec

In [30]:
with open('more_chunks', 'w') as json_file:
    json.dump(processed_json, json_file, indent=4)

In [31]:
with open('chunks_3_through_189','r') as f1:
    book_one = json.load(f1)

with open('more_chunks', 'r') as f2:
    book_two = json.load(f2)

combined_data = book_one + book_two

with open('first_batch_total', 'w') as f_combined:
    json.dump(combined_data, f_combined, indent=4)